# Yelp Data Challenge - NLP

BitTiger DS501

Jun 2017

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/last_2_years_restaurant_reviews.csv')

In [ ]:
df.head()

### Define your feature variables, here is the text of the review

In [ ]:
# Take the values of the column that contains review text data, save to a variable named "documents"
pass

In [ ]:
# inspect your documents, e.g. check the size, take a peek at elements of the numpy array
pass

### Define your target variable (any categorical variable that may be meaningful)

#### For example, I am interested in perfect (5 stars) and imperfect (1-4 stars) rating

In [ ]:
# Make a column and take the values, save to a variable named "target"
pass

#### You may want to look at the statistic of the target variable

In [ ]:
# To be implemented
pass

## Let's create training dataset and test dataset

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
# Documents is your X, target is your y
# Now split the data to training set and test set

In [ ]:
# Split to documents_train, documents_test, target_train, target_test
pass

## Let's get NLP representation of the documents

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create TfidfVectorizer, and name it vectorizer
pass

In [ ]:
# Train the model with your training data
pass

In [ ]:
# Get the vocab of your tfidf
pass

In [ ]:
# Use the trained model to transform your test data
pass

## Similar review search engine

In [ ]:
import numpy as np

# We will need these helper methods pretty soon

def get_top_values(lst, n, labels):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the highest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["cat", "pig"]
    '''
    return [labels[i] for i in np.argsort(lst)[::-1][:n]]  # np.argsort by default sorts values in ascending order

def get_bottom_values(lst, n, labels):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the lowest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["mouse", "rabbit"]
    '''
    pass  # To be implemented


In [ ]:
# Let's use cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Draw an arbitrary review from test (unseen in training) documents
pass

In [ ]:
# Transform the drawn review(s) to vector(s)
pass

In [ ]:
# Calculate the similarity score(s) between vector(s) and training vectors
pass

In [ ]:
# Let's find top 5 similar reviews
n = 5
pass

In [ ]:
print 'Our search query:'
print  # To be added

In [ ]:
print 'Most %s similar reviews:' % n
print  # To be added

#### Q: Does the result make sense to you?

A: (insert your comments here)

## Classifying positive/negative review

#### Naive-Bayes Classifier

In [ ]:
# Build a Naive-Bayes Classifier

from sklearn.naive_bayes import MultinomialNB

pass

In [ ]:
# Get score for training set
pass

In [ ]:
# Get score for test set
pass

#### Logistic Regression Classifier

In [ ]:
# Build a Logistic Regression Classifier

from sklearn.linear_model import LogisticRegression

pass

In [ ]:
# Get score for training set
pass

In [ ]:
# Get score for test set
pass

#### Q: What are the key features(words) that make the positive prediction?

In [ ]:
# Let's find it out by ranking
n = 20
pass

A: (insert your comments here)

#### Q: What are the key features(words) that make the negative prediction?

In [ ]:
# Let's find it out by ranking
n = 20
pass

A: (insert your comments here)

#### Random Forest Classifier

In [ ]:
# Build a Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier

pass

In [ ]:
# Get score for training set
pass

In [ ]:
# Get score for test set
pass

#### Q: What do you see from the training score and the test score?

A: (insert your comments here)

#### Q: Can you tell what features (words) are important by inspecting the RFC model?

In [ ]:
n = 20
pass

## Extra Credit #1: Use cross validation to evaluate your classifiers

[sklearn cross validation](http://scikit-learn.org/stable/modules/cross_validation.html)

In [ ]:
# To be implemented
pass

## Extra Credit #2: Use grid search to find best predictable classifier


[sklearn grid search tutorial (with cross validation)](http://scikit-learn.org/stable/modules/grid_search.html#grid-search)

[sklearn grid search documentation (with cross validation)](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)

In [ ]:
# To be implemented
pass